# Joint Measurement Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Joint Measurements kata](./JointMeasurements.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Joint Measurements kata (and this workbook):
1. Basic linear algebra
2. The concept of qubit and multi-qubit systems
3. Single-qubit and multi-qubit quantum gates and using them to manipulate the state of the system
4. The concept of measurements

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.12.20082513

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


In [ ]:
%workspace reload

### Task 1. Single-qubit measurement

**Input:** Two qubits (stored in an array) which are guaranteed to be either in superposition of states $|00\rangle$ and $|11\rangle$ or in superposition of states $|01\rangle$ and $|10\rangle$.

**Output:**  0 if qubits were in the first superposition, 1 if they were in the second superposition.  
*The state of the qubits at the end of the operation does not matter.*

### Solution

This task is a special case of the [task 1.9 of Measurements kata](./../Measurements/Measurements.ipynb#Task-1.9.-Distinguish-two-superposition-states-given-by-two-arrays-of-bit-strings), so we can use the same approach here (although much simpler, since we need to distinguish two specific superpositions of bit strings).

When we measure all qubits of a certain superposition state, it collapses to one of the basis vectors that comprised the superposition. 
We can do exactly that: measure both qubits and see whether the measurement results are the same; if they are, the state was a superposition of $|00\rangle$ and $|11\rangle$, otherwise the state was a superposition of $|01\rangle$ and $|10\rangle$.

In [ ]:
%kata T01_SingleQubitMeasurement_Test
open Microsoft.Quantum.Measurement;

operation SingleQubitMeasurement (qs : Qubit[]) : Int {
    return M(qs[0]) == M(qs[1]) ? 0 | 1;
}

[Return to task 1 of the Joint Measurements kata.](./JointMeasurements.ipynb#Task-1.-Single-qubit-measurement)

### Task 2. Parity measurement

**Inputs**: Two qubits (stored in an array) which are guaranteed to be either in superposition of states $|00\rangle$ and $|11\rangle$ or in superposition of states $|01\rangle$ and $|10\rangle$.

**Output**: 0 if qubits were in the first superposition, 1 if they were in the second superposition.  
*The state of the qubits at the end of the operation should be the same as the starting state.*

### Solution
This task is the same as the previous task, except that we are asked to revert back to the initial state after measurement. However, if we try to measure both qubits again, we'll lose the information about the original superposition states: a state $\alpha |00\rangle + \beta |11\rangle$ will collapse to either $|00\rangle$ or $|11\rangle$, and we won't be able to recover the information about the coefficients $\alpha$ and $\beta$.

We need to measure *parity* of the state without collapsing it all the way to the basis states. [Pauli measurements](https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements) can be used for joint measurements involving multiple qubits. For this task we apply the `PauliZ` matrix on both qubits.

A joint measurement using `PauliZ` can be thought as projecting the measured state to one of the two eigenspaces of `PauliZ` with $+1$ and $-1$ as the corresponding eigenvalues. The measurement returns `Zero` if the measured state is projected to the space with an eigenvalue of $+1$, and a result of `One` if projected to the space with an eigenvalue of $-1$.

To solidify this intuition, let us see what happens to various two-qubit states after joint measurement. The `PauliZ` matrix for two qubits is denoted by $Z \otimes Z$: 

$$Z \otimes Z = \begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & -1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix}$$


When this transformation is applied to a basis state $|00\rangle$, we get

$$\begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & -1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix} 
\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \\ \end{bmatrix} = 
\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \\ \end{bmatrix}$$

Comparing this to the characteristic equation for eigenvectors of $Z \otimes Z$ given by
$ Z \otimes Z |\psi\rangle = \lambda |\psi\rangle$,
it is easy to see that $|00\rangle$ belongs to the $+1$ eigenspace, hence the $Z \otimes Z$ measurement will return `Zero` and leave the state unchanged.

Similarly, it can easily be verified that $|11\rangle$ also belongs to $+1$ eigenspace, while $|01\rangle$ and $|10\rangle$ belong to the $-1$ eigenspace.

Now, what happens if we apply a $Z \otimes Z$ measurement to a superposition state $\alpha |00\rangle + \beta |11\rangle$? We can see that 

$$\begin{bmatrix}
    1 & 0 & 0 & 0 \\
    0 & -1 & 0 & 0 \\
    0 & 0 & -1 & 0 \\
    0 & 0 & 0 & 1 \\
\end{bmatrix} 
\begin{bmatrix} \alpha \\ 0 \\ 0 \\ \beta \\ \end{bmatrix} = 
\begin{bmatrix} \alpha \\ 0 \\ 0 \\ \beta \\ \end{bmatrix}$$

So this state also belongs to the $+1$ eigenspace, and measuring it will return `Zero` and leave the state unchanged. Similarly, we can verify that an $\alpha |01\rangle + \beta |10\rangle$ state belongs to the $-1$ eigenspace, and measuring it will return `One` without changing the state.

Hence, based on the return value from a `PauliZ` joint measurement, we can recognize which of the superposition states we were given while preserving the initial superposition state.

In Q#, the operation [`Measure`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) can be used to measure multiple qubits using an array of [Pauli](https://docs.microsoft.com/en-us/quantum/user-guide/language/types?#primitive-types) constants that define the basis for measurement.

(This solution can also be used in task 1.)

In [ ]:
%kata T02_ParityMeasurement_Test

operation ParityMeasurement (qs : Qubit[]) : Int {
    return Measure([PauliZ, PauliZ], qs) == Zero ? 0 | 1;
}

[Return to task 2 of the Joint Measurements kata.](./JointMeasurements.ipynb#Task-2.-Parity-measurement)

### Task 3. $|0000\rangle + |1111\rangle$ or $|0011\rangle + |1100\rangle$  ?
**Inputs**: Four qubits (stored in an array) which are guaranteed to be either in superposition of states $|0000\rangle$ and $|1111\rangle$ or in superposition of states $|0011\rangle$ and $|1100\rangle$.

**Output** : 0 if qubits were in the first superposition, 1 if they were in the second superposition.  
*The state of the qubits at the end of the operation should be the same as the starting state.*

### Solution
If we measure the parity of all the qubits (using `PauliZ` joint measurement, similar to the previous task), we'll get the same measurement of `Zero` for both states. Notice, however, that the **2nd** and **3rd** qubits are the same in the first superposition ($|00\rangle$ or $|11\rangle$), while they are different in  the second superposition ($|01\rangle$ and $|10\rangle$).

Hence, measuring the parity of those two qubits in `PauliZ`, as before, helps us distinguish between the two superpositions without collapsing them. The operation [`Measure`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) will handle the task of measuring the parity of a subset of qubits.

In [ ]:
%kata T03_GHZOrGHZWithX_Test 

operation GHZOrGHZWithX (qs : Qubit[]) : Int {
    return Measure([PauliZ, PauliZ], qs[1 .. 2]) == Zero ? 0 | 1;
}

[Return to task 3 of the Joint Measurements kata.](./JointMeasurements.ipynb#Task-3.-$|0000\rangle-+-|1111\rangle$-or-$|0011\rangle-+-|1100\rangle$--?)

### Task 4. GHZ state or W state ?

**Inputs:** An **even** number of qubits (stored in an array) which are guaranteed to be either in a superposition of states  $|0..0\rangle$ and $|1..1\rangle$ (the [GHZ state](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state)) or in the [W state](https://en.wikipedia.org/wiki/W_state).

**Output:** 0 if qubits were in the first superposition, 1 if they were in the second superposition. 
*The state of the qubits at the end of the operation should be the same as the starting state.*

### Solution

Let's consider a measurement in $Z^{\otimes N}$ basis, i.e., a parity measurement on all $N$ qubits.

Since it is given that there is an **even** number of qubits, both basis states the first superposition have parity $0$ and belong to the $+1$ eigenspace of this operator, and measuring them will return `Zero`. Each basis state of the second superposition always has a single qubit in the $|1\rangle$ state, and thus has parity $1$ and is guaranteed to be projected to the $-1$ eigenspace, returning `One`.

Q# operation [MeasureAllZ](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.measureallz) is a convenient shorthand for operation `Measure` with an argument that consists of all `PauliZ` bases.

In [ ]:
%kata T04_GHZOrWState_Test
open Microsoft.Quantum.Measurement;

operation GHZOrWState (qs : Qubit[]) : Int {
    return MeasureAllZ(qs) == Zero ? 0 | 1;
}

[Return to task 4 of the Joint Measurements kata.](./JointMeasurements.ipynb#Task-4.-GHZ-state-or-W-state-?)

### <a name="parity-x"></a> Task 5*. Parity measurement in different basis

**Inputs:** Two qubits (stored in an array) which are guaranteed to be
* either in superposition $\alpha |00\rangle + \beta |01\rangle + \beta |10\rangle + \alpha |11\rangle$
* or in superposition $\alpha |00\rangle - \beta |01\rangle + \beta |10\rangle - \alpha |11\rangle$

**Output:** 0 if qubits were in the first superposition, 1 if they were in the second superposition.  
*The state of the qubits at the end of the operation should be the same as the starting state.*

### Solution

Measuring the parity of these states in `PauliZ` basis is not going to help, since each state has all four basis states represented in it, the difference is only in the relative phases. 
Let's follow the hint given by the title and try to rewrite these states in a different basis - the Hadamard basis $\{|+\rangle, |-\rangle\}$.

The gives states could be regrouped as follows:

$$|\psi_1\rangle = \alpha (|00\rangle + |11\rangle) + \beta (|01\rangle + |10\rangle) \\
|\psi_2\rangle = \alpha (|00\rangle - |11\rangle) + \beta (- |01\rangle + |10\rangle)$$

The two-qubit basis states in the Hadamard basis are the following:

$$
|++\rangle = \frac12(|00\rangle + |11\rangle + |01\rangle + |10\rangle)\\
|--\rangle = \frac12(|00\rangle + |11\rangle - |01\rangle - |10\rangle)\\
|+-\rangle = \frac12(|00\rangle - |11\rangle - |01\rangle + |10\rangle)\\
|-+\rangle = \frac12(|00\rangle - |11\rangle + |01\rangle - |10\rangle)$$

Now it's easy to see how to represent the terms of our given states in the Hadamard basis:

$$
|00\rangle + |11\rangle = |++\rangle + |--\rangle\\
|01\rangle + |10\rangle = |++\rangle - |--\rangle\\
|00\rangle - |11\rangle = |+-\rangle + |-+\rangle\\
|10\rangle - |01\rangle = |+-\rangle - |-+\rangle$$

You can switch between the computational basis and the Hadamard basis using the H gate: 

$$|00\rangle + |11\rangle = |++\rangle + |--\rangle = H^{\otimes 2} (|00\rangle + |11\rangle)$$

and so on, where $H^{\otimes 2} = H \otimes H$ means applying the Hadamard operator to both qubits.

Therefore, we can represent our starting states as 

$$|\psi_1\rangle = \alpha H^{\otimes 2} (|00\rangle + |11\rangle) + \beta H^{\otimes 2} (|00\rangle - |11\rangle) = (\alpha + \beta) H^{\otimes 2} |00\rangle + (\alpha - \beta) H^{\otimes 2}|11\rangle$$

and, similarly,

$$|\psi_2\rangle = (\alpha + \beta) H^{\otimes 2} |01\rangle + (\alpha - \beta) H^{\otimes 2}|10\rangle$$

We now have a familiar format where the first superposition $|\psi_1\rangle$ has a parity of 0 and the second superposition $|\psi_2\rangle$ has a parity of 1, when considered in Hadamard basis.

We can implement the necessary measurement manually, by applying the H gate to each of the qubits, measuring the parity of the state similarly to [task 2](#Task-2.-Parity-measurement) and applying the H gate to each of the qubits again. 

It is easier and more neat to use the `Measure` operation with [PauliX](https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements) basis which performs an equivalent measurement.

In [ ]:
%kata T05_DifferentBasis_Test
open Microsoft.Quantum.Measurement;

operation DifferentBasis (qs : Qubit[]) : Int {
    return Measure([PauliX, PauliX], qs) == Zero ? 0 | 1;
}

[Return to task 5 of the Joint Measurements kata.](./JointMeasurements.ipynb#parity-x)

### <a name="controlled-x-0"></a> Task 6*. Controlled X gate with $|0\rangle$ target

**Input:** Two unentangled qubits (stored in an array of length 2). The first qubit (`qs[0]`) will be in state $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ , the second (`qs[1]`) - in state $|0\rangle$ (this can be written as two-qubit state $\big(\alpha |0\rangle + \beta |1\rangle\big) \otimes |0\rangle$). 

**Output:** Change the two-qubit state to $\alpha |00\rangle + \beta |11\rangle$ using only single-qubit gates and joint measurements. Do not use two-qubit gates. You do not need to allocate extra qubits.

### Solution

Lets apply $H$ to the second qubit. This results in

$$\big(\alpha |0\rangle + \beta |1\rangle\big) \otimes \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$$

A key point to note is that at this point the two qubits are **unentangled**. However, a joint measurement in `PauliZ` basis causes the qubits to become entangled and collapse into one of the following states:

* $ \alpha |00\rangle +  \beta |11\rangle$ if the measurement result was `Zero`
* or $\alpha |01\rangle + \beta |10\rangle$ if the measurement result was `One`.

If the qubits end up collapsing to the second state (given by the value of the measurement), applying the $X$ gate on the second qubit allows to fix the outcome to produce the desired result.

In [ ]:
%kata T06_ControlledX_Test
open Microsoft.Quantum.Measurement;

operation ControlledX (qs : Qubit[]) : Unit {
    H(qs[1]);
    let m = Measure([PauliZ, PauliZ], qs);
    
    if (m == One) {
        X(qs[1]);
    }
}

[Return to task 6 of the Joint Measurements kata.](./JointMeasurements.ipynb#controlled-x-0)

### <a name="controlled-x"></a> Task 7**. Controlled X gate with arbitrary target

**Input:** Two qubits (stored in an array of length 2) in an arbitrary two-qubit state $\alpha |00\rangle + \beta |01\rangle + \color{blue}\gamma |10\rangle  + \color{blue}\delta |11\rangle$.

**Goal:** Change the two-qubit state to $\alpha |00\rangle + \beta |01\rangle + \color{red}\delta |10\rangle  + \color{red}\gamma |11\rangle$ using only single-qubit gates and joint measurements.  Do not use two-qubit gates.

> A general-case implementation of CNOT gate via joint measurements is described in [this paper](https://arxiv.org/pdf/1201.5734.pdf).

### Solution
The implementation of the circuit in page 3 as presented in the paper mentioned above is fairly straightforward. The table in page 4 describes the actions to perform on the control and target qubit depending on the measurements $P_{1}$, $P_{2}$ and $M$. $P_{1}$ and $P_{2}$ are joint measurements in `PauliZ` and $M$ is a single qubit `PauliZ` measurement.

For this task, it is necessary to allocate an ancillary qubit denoted by `anc`.

Initially, the input qubits are in superposition and the ancillary qubit is unentangled with the input qubits. Applying a Hadamard gate on the ancillary qubit gives,
$$(\alpha |00\rangle + \beta |01\rangle + \gamma |10\rangle + \delta |11\rangle) \otimes \frac{1}{\sqrt2}(|0\rangle + |1\rangle) $$
Note that even at this point, the ancillary and input qubits are **unentangled**.

Now the measurement $P_{1}$ is made. This causes the ancillary bit to become **entangled** with the input qubits.

$$\frac{1}{\sqrt2} \big(\alpha |\color{red}00\color{red}0\rangle + \alpha |\color{blue}00\color{blue}1\rangle + \beta |\color{red}01\color{red}0\rangle + \beta |\color{blue}01\color{blue}1\rangle + \gamma |\color{blue}10\color{blue}0\rangle  + \gamma |\color{red}10\color{red}1\rangle + \delta |\color{blue}11\color{blue}0\rangle + \delta |\color{red}11\color{red}1\rangle \big)$$

After a joint measurement $P_{1}$ in `PauliZ` between **qubit 1** and the **ancillary qubit**, the above state is projected to one of the $+1$ and $-1$ eigenstates.

* If the state is projected to the $+1$ eigenstate (measured as `Zero`), the resulting superposition is $\alpha |\color{red}00\color{red}0\rangle + \beta |\color{red}01\color{red}0\rangle + \gamma |\color{red}10\color{red}1\rangle + \delta |\color{red}11\color{red}1\rangle$

* If the state is projected to the $-1$ eigenstate (measured as `One`), the resulting superposition is $\alpha |\color{blue}00\color{blue}1\rangle + \beta |\color{blue}01\color{blue}1\rangle + \gamma |\color{blue}10\color{blue}0\rangle  + \delta |\color{blue}11\color{blue}0\rangle$ (note that in both cases the $\frac{1}{\sqrt2}$ coefficient is removed, since the resulting state must be normalized)

Let us assume $P_{1}$ was measured as `Zero`.

> The measurement in the paper follows a different convention: `Zero` would be `1` and `One` would be `0` in the paper. You have to keep this in mind when translating the paper into Q# code.

Now apply the Hadamard gate on the **second qubit** and the **ancillary qubit**. This yields the state,

$$\frac12 (\alpha |0\color{red}0\color{red}0\rangle + \alpha |0\color{blue}0\color{blue}1\rangle + \alpha |0\color{blue}1\color{blue}0\rangle + \alpha |0\color{red}1\color{red}1\rangle) + \\
\frac12 (\beta |0\color{red}0\color{red}0\rangle + \beta |0\color{blue}0\color{blue}1\rangle - \beta |0\color{blue}1\color{blue}0\rangle - \beta |0\color{red}1\color{red}1\rangle) + \\
\frac12 (\gamma |1\color{red}0\color{red}0\rangle - \gamma |1\color{blue}0\color{blue}1\rangle + \gamma |1\color{blue}1\color{blue}0\rangle - \gamma |1\color{red}1\color{red}1\rangle) + \\
\frac12 (\delta |1\color{red}0\color{red}0\rangle - \delta |1\color{blue}0\color{blue}1\rangle - \delta |1\color{blue}1\color{blue}0\rangle + \delta |1\color{red}1\color{red}1\rangle)$$

Now a joint measurement $P_{2}$ in `PauliZ` is applied between **qubit 2** and the **ancillary qubit**

* If the state is projected to the $+1$ eigenstate (measured as `Zero`), the resulting superposition is  
  $\big[(\alpha + \beta) |0\color{red}0\color{red}0\rangle + (\alpha - \beta) |0\color{red}1\color{red}1\rangle \big] + 
  \big[(\delta + \gamma) |1\color{red}0\color{red}0\rangle + (\delta - \gamma) |1\color{red}1\color{red}1\rangle\big]$

* If the state is projected to the $-1$ eigenstate (measured as `One`), the resulting superposition is  
  $\big[(\alpha + \beta) |0\color{blue}0\color{blue}1\rangle + (\alpha - \beta) |0\color{blue}1\color{blue}0\rangle\big] - 
  \big[(\delta + \gamma) |1\color{blue}0\color{blue}1\rangle + (\delta - \gamma) |1\color{blue}1\color{blue}0\rangle\big]$


Let us assume $P_{2}$ was measured as `Zero`. After applying the Hadamard gate to **qubit 2** and the **ancillary qubit**, we get

$$(\alpha + \beta) \frac12 (|000\rangle + |001\rangle + |010\rangle + |011\rangle) + \\
(\alpha - \beta) \frac12 (|000\rangle - |001\rangle - |010\rangle + |011\rangle) + \\
(\delta + \gamma)\frac12 (|100\rangle + |101\rangle + |110\rangle + |111\rangle) + \\
(\delta - \gamma)\frac12 (|100\rangle - |101\rangle - |110\rangle + |111\rangle)$$

After simplifying, it becomes

$$\big[\alpha|00\color{red}0\rangle + \beta|01\color{red}0\rangle + \delta|10\color{red}0\rangle + \gamma|11\color{red}0\rangle\big] + 
\big[\alpha|01\color{blue}1\rangle + \beta|00\color{blue}1\rangle + \delta|11\color{blue}1\rangle + \gamma|10\color{blue}1\rangle\big]$$

Finally, after measuring the **ancillary qubit** as $M$,

* If the measurement is `Zero`, we get $\alpha|00\color{red}0\rangle + \beta|01\color{red}0\rangle + \delta|10\color{red}0\rangle + \gamma|11\color{red}0\rangle$
  
  Hence if the three measured states are (`Zero`, `Zero`, `Zero`), we don't need to do anything further as the output is already in the desired state. Note that this would correspond to the first row (1, 1, 1) in the table in the paper.

* If the measurement is `One`, we get $\alpha|01\color{blue}1\rangle + \beta|00\color{blue}1\rangle + \delta|11\color{blue}1\rangle + \gamma|10\color{blue}1\rangle$
  
  Hence if the three measured states are (`Zero`, `Zero`, `One`), we don't need to do anything to the **first qubit** and apply the `X` gate to the **second** qubit to obtain the desired state. Note that this would correspond to the second row (1, 1, 0) in the table in the paper.

The rest of the table can be filled by a similar procedure, assuming different outcomes of the first two measurements and analyzing the further steps. Note that we need apply `Z` to the **first qubit** if $P_{2}$ is `One` and `X` to the **second qubit** if $P_{1}$ is not equal to $M$.

In [ ]:
%kata T07_ControlledX_General_Test
open Microsoft.Quantum.Measurement;

operation ControlledX_General (qs : Qubit[]) : Unit {
    using (anc = Qubit()) {
        H(anc);
        
        let c = qs[0];
        let t = qs[1];
        
        let p1 = MeasureAllZ([c, anc]);
        
        H(anc);
        H(t);
        
        let p2 = MeasureAllZ([anc, t]);
        
        H(anc);
        H(t);
        
        let m = MResetZ(anc);
        
        if (p2 == One) {
            Z(c);
        }
        
        if (p1 != m) {
            X(t);
        }
    }
}

[Return to task 7 of the Joint Measurements kata.](./JointMeasurements.ipynb#controlled-x)